### Compare Runs, Choose Model, and Deploy it to a REST API

- Run a hyperparameter sweep on a training script
- Compare the results of the runs in the MLFlow UI
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a Container image suitable for deployment to a cloud platform

In [55]:
import numpy as np 
import keras
import pandas as pd 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [56]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";"
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [57]:
# Training and Testing
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [58]:
# 
X_train = train.drop(columns=['quality'], axis=1).values
y_train = train['quality'].values.ravel()

X_test = test.drop(columns=['quality'], axis=1).values
y_test = test['quality'].values.ravel()


In [59]:
# Validation Set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.25, random_state=42)

signature = infer_signature(X_train, y_train)

In [60]:
# ANN Model

import mlflow.tensorflow


def train_model(params, epochs, X_train, y_train, X_val, y_val, X_test, y_test):

    # Model arachitecture
    mean = np.mean(X_train, axis=0) # mean of all columns in train set for normalization
    var = np.var(X_train, axis=0) # var of all columns in train set for normalization

    model = keras.Sequential([
        keras.Input([X_train.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(1)
    ])


    # Model Compilation
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=params['lr'], momentum=params['momentum']),
        loss='mean_squared_error',
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    # Train the ANN model with lr and momentum params with MLFlow tracking
    with mlflow.start_run(nested=True):
        
        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=64)

        eval_result = model.evaluate(X_val, y_val, batch_size=64)
        eval_rmse = eval_result[1]

        # Log the params
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)


        # Log the model 
        mlflow.tensorflow.log_model(model, "model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [61]:
def objective(params):
    # Start a nested run for each hyperopt evaluation
    with mlflow.start_run(nested=True):
    # MLFlow will track the parameters and results for each run

        return train_model(params, epochs=3, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)


In [62]:
space = {"lr":hp.loguniform("lr",np.log(1e-5), np.log(1e-1)), "momentum":hp.uniform('momentum', 0.0, 1.0)}

In [63]:
import mlflow.tensorflow
from hyperopt import fmin, tpe, Trials

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment('/wine-quality')

# Ensure no active runs exist
if mlflow.active_run():
    mlflow.end_run()

# Start a parent run for the hyperparameter optimization
with mlflow.start_run(run_name="hyperopt_search") as parent_run:
    trials = Trials()
    
    # Conduct the hyperparameter search using hyperopt
    best = fmin(
        fn=objective,  # train function
        space=space,   # all the parameters
        algo=tpe.suggest, 
        max_evals=4, 
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]

    # Log the best params, loss, and model to the parent run
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best rmse loss: {best_run['loss']}")

Epoch 1/3                                            

 1/44 ━━━━━━━━━━━━━━━━━━━━ 1:28 2s/step - loss: 30.8563 - root_mean_squared_error: 5.5548
11/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27.9033 - root_mean_squared_error: 5.2802 
24/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24.7594 - root_mean_squared_error: 4.9649
39/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 21.7600 - root_mean_squared_error: 4.6381
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 14.2260 - root_mean_squared_error: 3.7717 - val_loss: 4.9036 - val_root_mean_squared_error: 2.2144

Epoch 2/3                                            

 1/44 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - loss: 5.1150 - root_mean_squared_error: 2.2616
14/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.5168 - root_mean_squared_error: 2.1225 
23/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.2890 - root_mean_squared_error: 2.0681
32/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.1833 - root_mean_squared_error: 2.0428
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025/09/16 20:42:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run melodic-snake-808 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/7bef35ec27044ae79800797862ee2854

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593

🏃 View run sassy-stag-375 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/c8095472699d41c4ba3d084890ec7dfb

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593

Epoch 1/3                                                                      

 1/44 ━━━━━━━━━━━━━━━━━━━━ 36s 860ms/step - loss: 29.2147 - root_mean_squared_error: 5.4051
19/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.4667 - root_mean_squared_error: 5.4283   
32/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 29.5030 - root_mean_squared_error: 5.4316
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 29.0457 - root_mean_squared_error: 5.3894 - val_loss: 27.9038 - val_root_mean_squared_error: 5.2824

Epoch 2/3                                                                      

 1/44 ━━━

2025/09/16 20:42:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run awesome-gull-678 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/44af218a11794fcbbf77ba25ad8fd992

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593  

🏃 View run honorable-worm-409 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/e8b95f38e1364ca3bb04edbe3d5da5c5

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593  

Epoch 1/3                                                                      

 1/44 ━━━━━━━━━━━━━━━━━━━━ 24s 580ms/step - loss: 35.5081 - root_mean_squared_error: 5.9589
19/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.6532 - root_mean_squared_error: 5.8010   
42/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.4430 - root_mean_squared_error: 5.7829
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 32.9042 - root_mean_squared_error: 5.7362 - val_loss: 32.1119 - val_root_mean_squared_error: 5.6667

Epoch 2/3                                                                      

 1

...

🏃 View run languid-skink-389 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/425fadfc630f4395b9d264547777bdab

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593  

🏃 View run legendary-carp-33 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/331070d3b2b2470f9319ddec53a8b4c2

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593  

Epoch 1/3                                                                      

 1/44 ━━━━━━━━━━━━━━━━━━━━ 25s 589ms/step - loss: 33.4767 - root_mean_squared_error: 5.7859
19/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.4358 - root_mean_squared_error: 3.3773   
37/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.4707 - root_mean_squared_error: 2.7391 
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 3.1548 - root_mean_squared_error: 1.7762 - val_loss: 1.7867 - val_root_mean_squared_error: 1.3367

Epoch 2/3                                                                      

 1/4

2025/09/16 20:43:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run useful-mink-275 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/0fd22c41494040b48a413a0ac7036518

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593  

🏃 View run persistent-wolf-333 at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/1d190648c4ba4ef3bb069899d44e30da

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593  

100%|██████████| 4/4 [01:43<00:00, 25.86s/trial, best loss: 0.7901548743247986]


2025/09/16 20:43:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best parameters: {'lr': np.float64(0.020583072056206627), 'momentum': np.float64(0.4056010505429388)}
Best rmse loss: 0.7901548743247986
🏃 View run hyperopt_search at: http://127.0.0.1:5000/#/experiments/902051187291755593/runs/16ad25a745dd4394bcb3d0029bfe0256
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/902051187291755593


TODO: Load the model and make predictions

In [64]:
# Register the model in the model registery

